In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error  
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

***
앙상블 학습은 크게 독립적 앙상블과 의존적 앙상블로 나눌 수 있다.
***
***
독립적 앙상블 (Independent Ensemble)
***
독립적 앙상블 방법에서 사용하는 개별 분류기들은 서로 독립적인 알고리즘이므로 각 분류기는 서로 다른 머신러닝 알고리즘을 사용할 수 있다.  
각 분류기가 독립적이므로 병렬화 할 수 있다는 장점이 있다.  


***
의존적 앙상블 (Dependent Ensemble)
***
의존적 앙상블 방법은 독립적 앙상블 방법과는 달리 개별 학습기들이 서로 독립이 아닌 경우를 의미하며, 의존적 앙상블 방법 중 가장 유명한 방법은 부스팅이다.  



***
배깅(Bagging, Bootstrap AggrevatING)과 랜덤 포레스트(Random Forest)
***
배깅은 개별 분류기들의 결과를 종합하여 최종 분류기에 성능을 향상시키는 방법이다.   
개별 분류기들이 동일한 학습 데이터로 학습하는 보팅과는 달리, 배깅은 오리지널 학습 데이터셋에서 샘플(Bootstrap)을 뽑아서 학습한다.  
Bootstrap이란 중복을 허용한 랜덤 샘플링 방식을 의미하고, 개별 분류 모델의 결과값을 모아 다수결 투표를 통해 최종 예측하게 된다.  


랜덤 포레스트는 여러개의 개별 분류기인 의사결정 트리를 토대로 예측한 결과를 종합해 전체 예측 정확도를 높이는 방법이다.  

***
와인 데이터를 사용해 와인 종류를 분류하는 모델을 생성하고 학습시킨다.
***

In [2]:
# 데이터 불러오기
raw_data = datasets.load_wine() # 사이킷런이 제공하는 와인 데이터를 불러온다.

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)


# 학습 데이터와 테스트 데이터로 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화된 스케일러에 적용한다.

# 모델 생성 후 데이터 학습
from sklearn.ensemble import RandomForestClassifier # 랜덤 포레스트 알고리즘 사용

# 앙상블의 랜덤 포레스트도 의사결정 트리를 내부적으로 사용하는 방법이므로 과적합 방지를 위해 
# max_depth 속성과 동일한 트리를 만들 수 있도록 random_state 속성을 지정해서 모델을 생성한다.
clf = RandomForestClassifier(max_depth=4, random_state=0)

# 표준화된 학습 데이터 x_train와 학습 데이터에 따른 레이블 데이터 y_train으로 앙상블 랜덤 포레스트 모델을 학습시킨다.
clf.fit(x_train, y_train)


(178, 13) (178,)
(133, 13) (45, 13) (133,) (45,)


RandomForestClassifier(max_depth=4, random_state=0)

***
학습된 모델로 테스트 데이터를 예측한다.
***

In [3]:
#predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict = clf.predict(x_test)
print(predict)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


***
학습된 모델을 평가한다
***

In [4]:
# 혼동 행렬
conf_matrix = confusion_matrix(y_test, predict)
print(conf_matrix)

[[16  0  0]
 [ 0 20  1]
 [ 0  0  8]]


In [5]:
#분류 리포트
class_report = classification_report(y_test, predict, target_names=raw_data.target_names)
print(class_report)

              precision    recall  f1-score   support

     class_0       1.00      1.00      1.00        16
     class_1       1.00      0.95      0.98        21
     class_2       0.89      1.00      0.94         8

    accuracy                           0.98        45
   macro avg       0.96      0.98      0.97        45
weighted avg       0.98      0.98      0.98        45

